# ***Context-Aware Chatbot Using LangChain or RAG***

In [1]:
# Add 'langchain-text-splitters' to your install list
!pip install -q langchain langchain-community langchain-text-splitters chromadb wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
!pip install -q -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 5.0 MB/s eta 0:00:00


In [3]:
!pip install -q -U langchain-classic

In [34]:
!pip install -q streamlit
!npm install -g localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.2 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 3s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙npm notice
npm notice New major version of npm available! 10.8.2 -> 11.7.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.7.0
npm notice To update run: npm install -g npm@11.7.0
npm notice
⠙

## ***Import Libraries***

In [42]:
# to handle data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

import os
from google.colab import userdata
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.prompts import PromptTemplate

from langchain_openai import ChatOpenAI

import streamlit as st


# ignore warnings

import warnings
warnings.filterwarnings('ignore')



## ***Load Content from Wikipedia***

In [5]:
# Search and load a specific topic from Wikipedia
# load_max_docs=1 ensures we get the most relevant page
topic = "Generative artificial intelligence"
loader = WikipediaLoader(query=topic, load_max_docs=2)
docs = loader.load()

print(f"Loaded {len(docs)} pages from Wikipedia.")
print(f"Source: {docs[0].metadata['source']}")

Loaded 2 pages from Wikipedia.
Source: https://en.wikipedia.org/wiki/Generative_artificial_intelligence


In [6]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks = text_splitter.split_documents(docs)



In [7]:
# Initialize HuggingFace Embeddings (Free and runs on Colab GPU/CPU)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Create Vector Store (Chroma)
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)
print("Vector database created successfully.")


Vector database created successfully.


In [25]:
# Set your OpenRouter API Key
os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')


In [26]:
# Initialize using ChatOpenAI (the OpenRouter "bridge")
llm = ChatOpenAI(
    model="xiaomi/mimo-v2-flash:free", # Note the "google/" prefix
    openai_api_key=os.environ["OPENROUTER_API_KEY"],
    openai_api_base="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "https://colab.research.google.com/", # Required by OpenRouter
        "X-Title": "My LangChain RAG Bot",
    },
    temperature=0.3
)


In [27]:
# Create a Custom System Prompt
# This ensures the bot only uses Wikipedia and doesn't guess.
custom_template = """
You are a helpful AI assistant. Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always provide a concise and professional response.

Context: {context}
Chat History: {chat_history}
Question: {question}

Helpful Answer:"""

CUSTOM_PROMPT = PromptTemplate(
    template=custom_template,
    input_variables=["context", "chat_history", "question"]
)


In [28]:
# 4. Setup Memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer' # Important for Gemini chains
)

In [29]:
# 5. Build the Conversational Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}), # Retrieve top 5 chunks
    memory=memory,
    combine_docs_chain_kwargs={"prompt": CUSTOM_PROMPT}
)

print("RAG Chatbot is live!")


Gemini RAG Chatbot is live!


In [31]:
def chat_with_bot():
    print(f"--- Chatbot is ready! Topic: {topic} ---")
    print("(Type 'exit' to stop)")

    while True:
        query = input("You: ")
        if query.lower() in ["exit", "quit", "stop"]:
            break

        result = qa_chain.invoke({"question": query})
        print(f"Bot: {result['answer']}\n")



In [33]:
chat_with_bot()

--- Chatbot is ready! Topic: Generative artificial intelligence ---
(Type 'exit' to stop)
You: World Cup
Bot: Based on the provided context, there is no mention of the World Cup or any connection between it and AI.

You: stop


## ***Streamlit***

In [36]:
%%writefile app.py

# --- UI CONFIG ---
st.title("🦜 Wikipedia RAG Bot")

# --- LOAD API KEY (Using Colab Secrets logic for Streamlit) ---
# Note: In a real deployment, use st.secrets. In Colab, we can use os.environ
api_key = os.getenv("OPENROUTER_API_KEY")

if not api_key:
    st.error("API Key not found. Please run the environment setup cell first.")
    st.stop()

# --- MODEL SETUP ---
if "llm" not in st.session_state:
    st.session_state.llm = ChatOpenAI(
        model="google/gemini-2.0-flash",
        api_key=api_key,
        base_url="https://openrouter.ai/api/v1",
        default_headers={"HTTP-Referer": "https://colab.research.google.com/"}
    )

# --- CHAT INTERFACE ---
if "messages" not in st.session_state:
    st.session_state.messages = []

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input():
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)

    # Simple placeholder logic (Connect your qa_chain here)
    response = f"I received: {prompt}. (Connect your vectorstore to get RAG answers!)"

    st.session_state.messages.append({"role": "assistant", "content": response})
    st.chat_message("assistant").write(response)

Writing app.py


In [45]:
import urllib
print("Password/Endpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))


Password/Endpoint IP for localtunnel is: 34.124.224.238


In [47]:
# 1. Kill any existing streamlit processes
!pkill streamlit

# 2. Run Streamlit with specific "Headless" and "CORS" flags
!streamlit run app.py --server.headless true --server.enableCORS false --server.enableXsrfProtection false &>/content/logs.txt &

# 3. Expose it again
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧your url is: https://icy-pens-buy.loca.lt
^C
